<a href="https://colab.research.google.com/github/Phimphika113/BSC_DPDM23/blob/main/weekly_work_neos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install -q pyomo ## ติดตั้ง Pyomo
from pyomo.environ import *
from pyomo import environ as pe

!sudo apt-get install  glpk-utils
solver = pe.SolverFactory('glpk', executable='/user/bin/glpsol')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


Failed to set executable for solver glpk. File with name=/user/bin/glpsol either does not exist or it is not executable. To skip this validation, call set_executable with validate=False.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyomo/opt/base/solvers.py", line 148, in __call__
    opt = self._cls[_name](**kwds)
  File "/usr/local/lib/python3.10/dist-packages/pyomo/solvers/plugins/solvers/GLPK.py", line 92, in __init__
    SystemCallSolver.__init__(self, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyomo/opt/solver/shellcmd.py", line 66, in __init__
    self.set_executable(name=executable, validate=validate)
  File "/usr/local/lib/python3.10/dist-packages/pyomo/opt/solver/shellcmd.py", line 115, in set_executable
    raise ValueError(
ValueError: Failed to set executable for solver glpk. File with name=/user/bin/glpsol either does not exist or it is not executable. To skip this validation, call set_executable with validate=False.


In [19]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [20]:
# Define days (1 week)
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

# Enter shifts of each day
shifts = ['morning', 'evening', 'night']  # 3 shifts of 8 hours
days_shifts = {day: shifts for day in days}  # dict with day as key and list of its shifts as value

# Enter workers ids (name, number, ...)
workers = ['W' + str(i) for i in range(1, 11)]  # 10 workers available, more than needed


In [21]:
# Initialize model
model = ConcreteModel()

# binary variables representing if a worker is scheduled somewhere
model.works = Var(((worker, day, shift) for worker in workers for day in days for shift in days_shifts[day]),
                  within=Binary, initialize=0)

# binary variables representing if a worker is necessary
model.needed = Var(workers, within=Binary, initialize=0)

# binary variables representing if a worker worked on sunday but not on saturday (avoid if possible)
model.no_pref = Var(workers, within=Binary, initialize=0)

In [22]:
# Define an objective function with model as input, to pass later
def obj_rule(m):
    c = len(workers)
    return sum(m.no_pref[worker] for worker in workers) + sum(c * m.needed[worker] for worker in workers)
# we multiply the second term by a constant to make sure that it is the primary objective
# since sum(m.no_prefer) is at most len(workers), len(workers) + 1 is a valid constant.


# add objective function to the model. rule (pass function) or expr (pass expression directly)
model.obj = Objective(rule=obj_rule, sense=minimize)

In [36]:
model.constraints = ConstraintList()  # Create a set of constraints

# Constraint: all shifts are assigned
for day in days:
    for shift in days_shifts[day]:
        if day in days[:-1] and shift in ['morning', 'evening']:
            # weekdays' and Saturdays' day shifts have exactly two workers
            model.constraints.add(  # to add a constraint to model.constraints set
                2 == sum(model.works[worker, day, shift] for worker in workers)
            )
        else:
            # Sundays' and nights' shifts have exactly one worker
            model.constraints.add(
                1 == sum(model.works[worker, day, shift] for worker in workers)
            )

# Constraint: no more than 40 hours worked
for worker in workers:
    model.constraints.add(
        40 >= sum(8 * model.works[worker, day, shift] for day in days for shift in days_shifts[day])
    )

# Constraint: rest between two shifts is of 12 hours (i.e., at least two shifts)
for worker in workers:
    for j in range(len(days)):
        # if working in morning, cannot work again that day
        model.constraints.add(
            1 >= sum(model.works[worker, days[j], shift] for shift in days_shifts[days[j]])
        )
        # if working in evening, until next evening (note that after sunday comes next monday)
        model.constraints.add(
            1 >= sum(model.works[worker, days[j], shift] for shift in ['evening', 'night']) +
            model.works[worker, days[(j + 1) % 7], 'morning']
        )
        # if working in night, until next night
        model.constraints.add(
            1 >= model.works[worker, days[j], 'night'] +
            sum(model.works[worker, days[(j + 1) % 7], shift] for shift in ['morning', 'evening'])
        )

# Constraint (def of model.needed)
for worker in workers:
    model.constraints.add(
        10000 * model.needed[worker] >= sum(model.works[worker, day, shift] for day in days for shift in days_shifts[day])
    )  # if any model.works[worker, ·, ·] non-zero, model.needed[worker] must be one; else is zero to reduce the obj function
    # 10000 is to remark, but 5 was enough since max of 40 hours yields max of 5 shifts, the maximum possible sum

# Constraint (def of model.no_pref)
for worker in workers:
    model.constraints.add(
        model.no_pref[worker] >= sum(model.works[worker, 'Sat', shift] for shift in days_shifts['Sat'])
        - sum(model.works[worker, 'Sun', shift] for shift in days_shifts['Sun'])
    )  # if not working on sunday but working saturday model.needed must be 1; else will be zero to reduce the obj function

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [37]:
import os
# Set the NEOS email address directly in the environment
os.environ['NEOS_EMAIL'] = "your_actual_email@example.com"  # Replace with your actual email

opt = SolverFactory('cbc')  # Select solver
solver_manager = SolverManagerFactory('neos')  # Solve in neos server
results = solver_manager.solve(model, opt=opt)

In [38]:
print(results)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 261
  Number of variables: 230
  Sense: unknown
Solver: 
- Status: ok
  Message: CBC 2.10.10 optimal, objective 72; 0 nodes, 88 iterations, 0.098926 seconds
  Termination condition: optimal
  Id: 0
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [33]:
def get_workers_needed(needed):
    """Extract to a list the needed workers for the optimal solution."""
    workers_needed = []
    for worker in workers:
        if needed[worker].value == 1:
            workers_needed.append(worker)
    return workers_needed


def get_work_table(works):
    """Build a timetable of the week as a dictionary from the model's optimal solution."""
    week_table = {day: {shift: [] for shift in days_shifts[day]} for day in days}
    for worker in workers:
        for day in days:
            for shift in days_shifts[day]:
                    if works[worker, day, shift].value == 1:
                        week_table[day][shift].append(worker)
    return week_table


def get_no_preference(no_pref):
    """Extract to a list the workers not satisfied with their weekend preference."""
    return [worker for worker in workers if no_pref[worker].value == 1]


workers_needed = get_workers_needed(model.needed)  # dict with the optimal timetable
week_table = get_work_table(model.works)  # list with the required workers
workers_no_pref = get_no_preference(model.no_pref)  # list with the non-satisfied workers (work on Sat but not on Sun)


In [39]:
import json

In [40]:
print('Workers needed:')
[print('  ' + worker) for worker in workers_needed];

Workers needed:
  W2
  W3
  W5
  W7
  W8
  W9
  W10


In [41]:
print('Work schedule:')
print(json.dumps(week_table, indent=2))

Work schedule:
{
  "Mon": {
    "morning": [
      "W3",
      "W9"
    ],
    "evening": [
      "W8",
      "W10"
    ],
    "night": [
      "W2"
    ]
  },
  "Tue": {
    "morning": [
      "W3",
      "W5"
    ],
    "evening": [
      "W8",
      "W10"
    ],
    "night": [
      "W9"
    ]
  },
  "Wed": {
    "morning": [
      "W5",
      "W7"
    ],
    "evening": [
      "W2",
      "W10"
    ],
    "night": [
      "W3"
    ]
  },
  "Thu": {
    "morning": [
      "W7",
      "W9"
    ],
    "evening": [
      "W8",
      "W10"
    ],
    "night": [
      "W2"
    ]
  },
  "Fri": {
    "morning": [
      "W7",
      "W9"
    ],
    "evening": [
      "W3",
      "W10"
    ],
    "night": [
      "W5"
    ]
  },
  "Sat": {
    "morning": [
      "W2",
      "W7"
    ],
    "evening": [
      "W8",
      "W9"
    ],
    "night": [
      "W3"
    ]
  },
  "Sun": {
    "morning": [
      "W7"
    ],
    "evening": [
      "W8"
    ],
    "night": [
      "W2"
    ]
  }
}


In [42]:
print('Workers not satisfied by weekend condition:')
[print('  ' + worker) for worker in workers_no_pref];

Workers not satisfied by weekend condition:
  W3
  W9


In [43]:
print('Objective value:', model.obj())

Objective value: 72.0
